In [212]:
from Loss import loss_func
import pandas as pd
from scipy.optimize import minimize
import numpy as np

In [243]:
def save_loss(dist: float, road: str, inflow) -> float:
    """
    Считет значение loss_func, если его раньше не считали
    """
    #
    if (type(dist) == np.ndarray):  # не трогать блять, иначе сраный minimize сломет все, ебанная он сука, вот нахуя вместо числа подставлять массив с этим числом, он что тупой? сука драная, я полжизни потратил на поиск ебанной ошибки
        dist = dist[0]
    #

    # проверяем: вдруг мы это уже считали
    filename = "Loss_res/check.csv"
    loss_res = pd.read_csv(filename)
    cond = True
    for i in range(len(inflow)):
        cond = cond & (loss_res.iloc[:, i] == inflow[i])
    cond = cond & (loss_res['dist'] == dist)
    loss = loss_res[cond]['loss']  # все элементы с данным inlfow и dist
    assert len(loss) <= 1, f"Ну пизда, и что делать?\n{loss_res[cond]}"

    # eсли не считали, то считаем
    if (len(loss) == 0):
        loss = loss_func(dist, road, inflow)
        new_row = (*inflow, dist, loss)
        loss_res.loc[len(loss_res)] = new_row
        loss_res.to_csv(filename, index=False)  # не задавайте вопросы про скорость и оптимизацию, оно, вроде, работает
        return loss
    return loss.iloc[0]

# Тупая оптимизация

In [244]:
def dumb_best_dist(road: str, inflow) -> float:
    ans = minimize(save_loss, 2.0, args=(road, inflow), bounds=[(0.0, None)])
    print(ans)

dumb_best_dist('road_name', (1, 2, 1))

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 1.0
        x: [ 1.000e+00]
      nit: 1
      jac: [ 0.000e+00]
     nfev: 4
     njev: 2
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>


# Что-то на ML или DL жду Ваших предложений